In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install -U ultralytics==8.3.32


In [1]:
# ============================================================
# BLOCK 1 — CONFIG, IMPORTS, SEEDING
# ============================================================

import os
import json
import math
import random
import gc
from pathlib import Path
from collections import defaultdict

import cv2
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import models
from sklearn.model_selection import GroupKFold
from tqdm.auto import tqdm

# ---------------- CONFIG ----------------
CFG = {
    "input_root": Path("/kaggle/input/face-sideview-kp/train"),
    "coco_json": Path("/kaggle/input/face-sideview-kp/train/_annotations.coco.json"),
    "yolo_weights": Path("/kaggle/input/yolov11n-seg-weights/pytorch/default/1/yolo11n-seg.pt"),

    "work_root": Path("/kaggle/working/final_clean_run"),
    "img_size": 512,
    "hm_size": 64,
    "num_kpts": 26,

    "batch_size": 32,
    "epochs": 20,
    "lr": 3e-4,
    "weight_decay": 1e-5,

    "seed": 42,
    "folds": 5,
    "device": torch.device("cuda" if torch.cuda.is_available() else "cpu"),
}

CFG["work_root"].mkdir(parents=True, exist_ok=True)

# ---------------- SEEDING ----------------
def seed_everything(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

seed_everything(CFG["seed"])

print("Device:", CFG["device"])
print("Work dir:", CFG["work_root"])


Device: cpu
Work dir: /kaggle/working/final_clean_run


In [2]:
# ============================================================
# BLOCK 2 — LOAD & VALIDATE COCO
# ============================================================

with open(CFG["coco_json"], "r") as f:
    coco = json.load(f)

images_info = {im["id"]: im for im in coco["images"]}
anns_by_image = defaultdict(list)

for ann in coco["annotations"]:
    anns_by_image[ann["image_id"]].append(ann)

assert len(images_info) > 0, "No images found in COCO"
assert len(coco["annotations"]) > 0, "No annotations found in COCO"

KP_LEN = len(coco["annotations"][0]["keypoints"]) // 3
assert KP_LEN == CFG["num_kpts"], f"Expected {CFG['num_kpts']} keypoints, got {KP_LEN}"

# Sanity check: keypoints are absolute pixel coords
sample_ann = coco["annotations"][0]
xs = sample_ann["keypoints"][0::3]
ys = sample_ann["keypoints"][1::3]

assert max(xs) > 10 and max(ys) > 10, "Keypoints appear normalized — expected pixel coords"

print(f"Images: {len(images_info)}")
print(f"Annotations: {len(coco['annotations'])}")
print(f"Keypoints per instance: {KP_LEN}")


Images: 514
Annotations: 515
Keypoints per instance: 26


In [3]:
# ============================================================
# BLOCK 3 — YOLO SEGMENTATION & SAFE CROP
# ============================================================

from ultralytics import YOLO

assert CFG["yolo_weights"].exists(), "YOLO weights not found"

yolo = YOLO(str(CFG["yolo_weights"]))

def segment_person_black_bg(img_bgr):
    h, w = img_bgr.shape[:2]
    res = yolo(img_bgr, verbose=False, retina_masks=True)

    if not res or res[0].masks is None:
        out = img_bgr.copy()
        out[:] = 0
        return out

    r = res[0]
    masks = r.masks.data.cpu().numpy()
    boxes = r.boxes.cls.cpu().numpy()

    person_ids = [i for i, c in enumerate(boxes) if int(c) == 0]
    if not person_ids:
        out = img_bgr.copy()
        out[:] = 0
        return out

    best_i = max(
        person_ids,
        key=lambda i: masks[i].sum()
    )

    mask = masks[best_i]
    if mask.shape != (h, w):
        mask = cv2.resize(
            (mask > 0.5).astype(np.uint8),
            (w, h),
            interpolation=cv2.INTER_NEAREST
        )
    else:
        mask = (mask > 0.5).astype(np.uint8)

    out = img_bgr.copy()
    out[mask == 0] = 0
    return out

def square_crop_from_bbox(bbox, img_w, img_h, margin_ratio=0.25):
    x, y, bw, bh = bbox
    cx = x + bw / 2
    cy = y + bh / 2

    side = max(bw, bh)
    side *= (1.0 + 2 * margin_ratio)

    x1 = int(round(cx - side / 2))
    y1 = int(round(cy - side / 2))
    x2 = int(round(cx + side / 2))
    y2 = int(round(cy + side / 2))

    return x1, y1, x2, y2

def expand_crop_to_kps(crop, kps, img_w, img_h, margin=8):
    x1, y1, x2, y2 = crop

    xs = []
    ys = []
    for i in range(0, len(kps), 3):
        if kps[i+2] > 0:
            xs.append(kps[i])
            ys.append(kps[i+1])

    if not xs:
        return crop

    x1 = min(x1, int(min(xs) - margin))
    y1 = min(y1, int(min(ys) - margin))
    x2 = max(x2, int(max(xs) + margin))
    y2 = max(y2, int(max(ys) + margin))

    w = x2 - x1
    h = y2 - y1
    side = max(w, h)
    cx = x1 + w // 2
    cy = y1 + h // 2
    half = side // 2

    x1 = cx - half
    y1 = cy - half
    x2 = cx + half
    y2 = cy + half

    return (
        max(0, x1),
        max(0, y1),
        min(img_w, x2),
        min(img_h, y2),
    )


ModuleNotFoundError: No module named 'ultralytics'